# Extraction of models from DeciContas.br Dataset

In [1]:
import pandas as pd

from pprint import pprint
from langchain_openai import  AzureChatOpenAI
from dotenv import load_dotenv

from tools.prompt import generate_few_shot_ner_prompts
from tools.schema import (
    NERDecisao
)

load_dotenv()

gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="function_calling")

In [2]:
all_decisions = pd.read_csv("dataset/all_decisions.csv")

In [3]:
len(all_decisions)

40174

In [4]:
pd.to_datetime(all_decisions['datasessao']).dt.year

0        2016
1        2016
2        2016
3        2016
4        2016
         ... 
40169    2025
40170    2025
40171    2025
40172    2025
40173    2025
Name: datasessao, Length: 40174, dtype: int32

In [9]:
all_decisions['year'] = pd.to_datetime(all_decisions['datasessao']).dt.year

In [10]:
all_decisions_2023 = all_decisions[all_decisions['year'] >= 2023]

In [11]:
len(all_decisions_2023), len(all_decisions) 

(8728, 40174)

In [12]:
resultados = []
errors = []
for index, row in all_decisions_2023.iterrows():
    if index % 10 == 0 and index > 0:
        print(f"Processando instância {index} de {len(all_decisions_2023)}")
        all_decisions_2023.to_csv("dataset/all_decisions_2023_ner.csv", index=True)

    try:
        prompt_with_few_shot = generate_few_shot_ner_prompts(row['texto_acordao'])
        
        result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
        all_decisions_2023.at[index, 'ner_decisao'] = result
        resultados.append(result)
    except Exception as e:
        errors.append({
            'index': index,
            'error': str(e),
            'row': row.to_dict()
        })

/tmp/ipykernel_12742/1431655093.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_decisions_2023.at[index, 'ner_decisao'] = result


Processando instância 690 de 8728
Processando instância 700 de 8728
Processando instância 710 de 8728
Processando instância 720 de 8728
Processando instância 730 de 8728
Processando instância 740 de 8728
Processando instância 750 de 8728
Processando instância 760 de 8728
Processando instância 770 de 8728
Processando instância 780 de 8728
Processando instância 790 de 8728
Processando instância 800 de 8728
Processando instância 810 de 8728
Processando instância 820 de 8728
Processando instância 1360 de 8728
Processando instância 1370 de 8728
Processando instância 1380 de 8728
Processando instância 1390 de 8728
Processando instância 1400 de 8728
Processando instância 1410 de 8728
Processando instância 1420 de 8728
Processando instância 1430 de 8728
Processando instância 1440 de 8728
Processando instância 1450 de 8728
Processando instância 1460 de 8728
Processando instância 1470 de 8728
Processando instância 1480 de 8728
Processando instância 1490 de 8728
Processando instância 1500 de 8728

In [13]:
errors

[{'index': 683,
  'error': "object of type 'NERDecisao' has no len()",
  'row': {'datasessao': '2023-01-24 00:00:00.000',
   'texto_acordao': 'DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, concordando com a Informação do Corpo Técnico da DAP e com o Parecer do Ministério Público que atua junto a esta Corte de Contas em cumprimento aos comandos do Supremo Tribunal Federal em sede do referido Tema 445 de Repercussão Geral, e acolhendo integralmente o voto do Conselheiro Relator, julgar:\r\na)\tpelo REGISTRO tácito do ato concessivo de aposentadoria em apreço;\r\nb)\tapós o trânsito em julgado, ao Arquivo Geral. ',
   'idcomposicaopauta': 106564,
   'idvoto': 33657,
   'idvotopauta': 33657.0,
   'year': 2023}},
 {'index': 1657,
  'error': 'Function NERDecisao arguments:\n\n{"multas":[{"descricao_multa":"multa pessoal e diária de R$500,00 (quinhentos reais) à Secretária de Estado da Educação, da Cultura, do Esporte e do Lazer, nos termos do art. 110 da Lei Complem

In [14]:
for e in errors:
    prompt_with_few_shot = generate_few_shot_ner_prompts(e['row']['texto_acordao'])
    result = extractor_gpt4turbo.invoke(prompt_with_few_shot)
    all_decisions_2023.at[e['index'], 'ner_decisao'] = result
    resultados.append(result)

In [16]:
all_decisions_2023.to_pickle("dataset/all_decisions_2023_ner.pkl")

In [34]:
obrigacoes = [d.obrigacoes for d in all_decisions_2023['ner_decisao'] if d.obrigacoes != []]
nobrigacoes = [len(d) for d in obrigacoes]

In [36]:
sum(nobrigacoes)

1059

In [40]:
recomendacoes = [d.recomendacoes for d in all_decisions_2023['ner_decisao'] if d.recomendacoes != []]
nrecomendacoes = [len(d) for d in recomendacoes]

In [41]:
sum(nrecomendacoes)

683

In [38]:
all_decisions_2023.iloc[0]

datasessao                                     2023-01-24 00:00:00.000
texto_acordao        DECIDEM os Conselheiros do Tribunal de Contas ...
idcomposicaopauta                                               106564
idvoto                                                           33657
idvotopauta                                                    33657.0
year                                                              2023
ner_decisao          multas=[] ressarcimentos=[] obrigacoes=[] reco...
Name: 683, dtype: object

In [33]:
len(obrigacoes[1])

2